<a href="https://colab.research.google.com/github/pranav98711/Google-Colaboratory/blob/master/DeepLearning/MyProjects/Building_your_Deep_Neural_Network/Building_your_Deep_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from sample_data.dnn_utils import sigmoid, relu, relu_backward,sigmoid_backward
from sample_data.testcases import *  #just for checking ifeach helper functions created works properly of not

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

### Initialization

For 2 layer

In [0]:
def initialize_parameters(n_x,n_h,n_y):
  W1 = np.random.randn( n_x,n_h ) * 0.01
  b1=np.zeros((n_x,1))
  W2=np.random.randn( n_h,n_y ) * 0.01
  b2=np.zeros((n_y,1))


  parameters ={
      "W1" : W1,
      "b1" : b1,
      "W2" : W2,
      "b2" : b2
  }

  return parameters

In [0]:
#Testing
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

For L layer

In [0]:
def initialize_parameters_deep(layer_dims):
  np.random.seed(3)
  parameters={}
  L=len(layer_dims)
  for l in range(1,L):
    
    parameters["W" + str(l)] = np.random.randn( layer_dims[l] , layer_dims[l-1] ) * 0.01
    parameters["b" + str(l)] = np.zeros((layer_dims[l], 1 ))

  return parameters
    


In [0]:
#Testing
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

### Forward Propagation 

In [0]:
def linear_forward(A , W, b):
  Z=np.dot(W,A) + b
  cache=(A,W,b)
  return Z,cache

In [0]:
#Testing
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

In [0]:
def linear_activation_forward(A_prev , W,b,activation):
  Z,linear_cache=linear_forward(A_prev,W,b)

  if activation=="sigmoid":
    A,activation_cache=sigmoid(Z)

  elif activation=="relu":
    A,activation_cache=relu(Z)

  cache=(linear_cache,activation_cache)

  return A,cache

In [0]:
#Testing
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))


In [0]:
def L_model_forward(X,parameters):
  caches = []
  A = X
  L = len(parameters) //2   ##the floor division // rounds the result down to the nearest whole number

  for l in range(1,L):
    A_prev=A
    A,cache=linear_activation_forward(A_prev, parameters["W" + str(l)] ,parameters["b" + str(l)] , activation="relu"  )
    caches.append(cache)

  AL,cache=linear_activation_forward(A, parameters["W" + str(L)] ,parameters["b" + str(L)] , activation="sigmoid"  )
  caches.append(cache)
  
  return AL,caches

In [0]:
#Testing
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

## COmputing Cost

In [0]:
def compute_cost(AL , Y):
  m=Y.shape[1]
  cost = -(np.sum( np.multiply(Y , np.log(AL)) + np.multiply((1-Y) , np.log(1-AL))  ))/m
  cost=np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
  return cost



In [13]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.2797765635793423


## Backward propagation

In [0]:
def linear_backward(dZ,cache):
  A_prev,W,b = cache
  m=A_prev.shape[1]

  dW=np.dot(dZ , A_prev.T)/m
  db=np.sum(dZ , axis=1, keepdims=True)/m
  dA_prev=np.dot(W.T ,dZ)

  return dA_prev,dW,db

In [15]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]


In [0]:
def linear_activation_backward(dA,cache,activation):
  linear_cache,activation_cache = cache

  if activation=="relu":
    dZ = relu_backward(dA , activation_cache)
    dA_prev,dW,db=linear_backward(dZ,linear_cache)

  elif activation=="sigmoid":
    dZ=sigmoid_backward(dA,activation_cache)
    dA_prev,dW,db=linear_backward(dZ,linear_cache)

  return dA_prev,dW,db


In [20]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


In [0]:
def L_model_backward(AL , Y , caches):
  
  grads={}
  L=len(caches)
  m=AL.shape[1]
  Y=Y.reshape(AL.shape)

  dAL = -np.divide(Y,AL) + np.divide(1-Y , 1-AL)

  current_cache=caches[L-1]
  grads["dA" + str(L-1)] , grads["dW" + str(L)] , grads["db" + str(L)] = linear_activation_backward(dAL , current_cache, activation="sigmoid")


  for l in reversed(range(L-1)):
    current_cache =caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward( grads["dA" + str(l+1)], current_cache, activation="relu")
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l + 1)] = dW_temp
    grads["db" + str(l + 1)] = db_temp

  return grads

In [36]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


In [0]:
def update_parameters(parameters,grads,learning_rate):

  L = len(parameters) // 2 

  for l in range(L):
    parameters["W" + str(l+1)] =parameters["W" + str(l+1)] - learning_rate*grads["dW" + str(l+1)]  
    parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db" + str(l+1)] 
 
  return parameters

In [42]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
